In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = 'true'

from myapp.models import Datas

import numpy as np
import pandas as pd

In [2]:
from pydataset import data
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
label_encoder = preprocessing.LabelEncoder()

from mixed_naive_bayes import MixedNB
from tablib import Dataset

In [3]:
db = Datas.objects.all()
csv = pd.DataFrame(db.values())
df = pd.DataFrame(db.values())

In [4]:
df

,id,nama,nim,jk,ipk,ipk_prodi,p_cuti,sam_bekerja,a_papua,p_mengulang,ipk1,ipk2,ipk3,ipk4,ipk5,ipk6,prediksi
0,1,satu,23,l,2.4,2.4,y,y,y,y,2.0,2.0,2.0,2.0,2.0,2.0,t
1,2,dua,334,p,3.0,3.0,t,y,y,y,2.5,2.5,2.5,2.5,2.5,2.5,tw
2,3,tiga,343,l,3.2,3.2,y,y,y,t,3.3,3.3,3.3,3.3,3.3,3.3,tw
3,4,empat,233,l,2.0,2.0,y,y,y,t,3.0,3.0,3.0,3.0,3.0,3.0,tw
4,5,lima,344,p,2.4,2.4,t,t,y,t,3.0,3.0,3.0,3.0,3.0,3.0,tw
5,6,enam,223,p,2.5,2.5,t,t,t,y,2.8,2.8,2.8,2.8,2.8,2.8,t
6,7,tiju,234,p,3.0,3.0,y,t,t,t,2.7,2.7,2.7,2.7,2.7,2.7,tw
7,8,delapan,334,l,2.9,2.9,t,y,t,t,3.0,3.0,3.0,3.0,3.0,3.0,t
8,9,sembilan,553,l,2.7,2.7,t,y,y,y,3.0,3.0,3.0,3.0,3.0,3.0,tw


In [6]:
df.to_csv(r'db.sqlite3:\kkl.csv', index=False)

## ENDHEADER

In [5]:
df['ipk']

0    2.4
1    3.0
2    3.2
3    2.0
4    2.4
5    2.5
6    3.0
7    2.9
8    2.7
Name: ipk, dtype: float64

In [10]:
csv.columns

Index(['id', 'nama', 'nim', 'jk', 'ipk', 'ipk_prodi', 'p_cuti', 'sam_bekerja',
       'a_papua', 'p_mengulang', 'ipk1', 'ipk2', 'ipk3', 'ipk4', 'ipk5',
       'ipk6', 'prediksi'],
      dtype='object')

In [4]:
csv[['jk', 'p_mengulang', 'p_cuti', 'sam_bekerja','a_papua']] = csv[['jk', 'p_mengulang', 'p_cuti', 'sam_bekerja','a_papua']].apply(label_encoder.fit_transform)
csv[['ipk','ipk_prodi','ipk1', 'ipk2', 'ipk3', 'ipk4','ipk5', 'ipk6']] = np.where(csv[['ipk','ipk_prodi','ipk1', 'ipk2', 'ipk3', 'ipk4','ipk5', 'ipk6']] < 3, 0, 1)
y = csv['prediksi']	
x = csv[['jk', 'p_mengulang', 'ipk', 'p_cuti','sam_bekerja','a_papua',
	'ipk_prodi',
	'ipk1',
	'ipk2',
	'ipk3',
	'ipk4',
	'ipk5',
	'ipk6',
]]

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=3, random_state=0)
nv = GaussianNB()
ajar = nv.fit(x_train, y_train)
y_pred = nv.predict(x_test)

In [5]:
data_y = pd.DataFrame(y)
data_y = data_y.apply(label_encoder.fit_transform)
yyy = data_y['prediksi']
clf = MixedNB(categorical_features=[0,1])

In [9]:
clf.fit(x,yyy)

[2 2]


MixedNB(alpha=0.5, var_smoothing=1e-09)

In [8]:
prediksi_mixed = clf.predict(x_test)


In [10]:
encoder_y_test = label_encoder.transform(y_test)


In [11]:
akurasi_mixed = accuracy_score(encoder_y_test, prediksi_mixed)


In [12]:
gabung = x_train.copy()
gabung.insert(7,'prediksi',y_train,True)
x_train_y_train = gabung.copy()

In [13]:
gabung['jk'] = np.where(gabung['jk'] == 0, 'perempuan', 'laki-laki')
gabung[['sam_bekerja','p_mengulang', 'p_cuti', 'a_papua']] = np.where(gabung[['sam_bekerja','p_mengulang', 'p_cuti', 'a_papua']] == 0, 'tidak', 'ya')	# 2 kondisi ya/tidak
gabung['prediksi'] = np.where(gabung['prediksi'] == 'tw', 'Tepat Waktu', 'Terlambat')

In [20]:
kamusdata = {
		'jk'	: np.array([0]),
		'p_mengulang'	: np.array([0]),
		'ipk'	: np.array([0]),
		'p_cuti'	: np.array([0]),
		'sam_bekerja'	: np.array([0]),
		'a_papua'	: np.array([0]),
		'ipk_prodi'	: np.array([0]),
		'ipk1'	: np.array([0]),
		'ipk2'	: np.array([0]),
		'ipk3'	: np.array([0]),
		'ipk4'	: np.array([0]),
		'ipk5'	: np.array([0]),
		'ipk6'	: np.array([0]),
	}

test = pd.DataFrame(kamusdata)
prediksi_test = nv.predict(test)

# form = Fomrdata()

# prediksi_post = ''

# test['jk'] = np.where(test['jk'] == 0, ' ', ' ')
# test[['ipk','ipk1','sam_bekerja','p_mengulang', 'p_cuti', 'a_papua']] = np.where(test[['ipk','ipk1','sam_bekerja','p_mengulang', 'p_cuti', 'a_papua']] == 0, ' ', ' ')	# 2 kondisi ya/tidak


In [ ]:
csv[['jk', 'p_mengulang', 'p_cuti', 'sam_bekerja','a_papua']] = csv[['jk', 'p_mengulang', 'p_cuti', 'sam_bekerja','a_papua']].apply(label_encoder.fit_transform)

In [ ]:
csv[['ipk','ipk_prodi','ipk1', 'ipk2', 'ipk3', 'ipk4','ipk5', 'ipk6']] = np.where(csv[['ipk','ipk_prodi','ipk1', 'ipk2', 'ipk3', 'ipk4','ipk5', 'ipk6']] < 3, 0, 1)

In [ ]:
y = csv['prediksi']

In [ ]:
x = csv[['jk', 'p_mengulang', 'ipk', 'p_cuti','sam_bekerja','a_papua']]

In [ ]:
x = csv[['jk', 'p_mengulang', 'ipk', 'p_cuti','sam_bekerja','a_papua',
	'ipk_prodi',
	'ipk1',
	'ipk2',
	'ipk3',
	'ipk4',
	'ipk5',
	'ipk6',
]]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=0)
nv = GaussianNB()

ajar = nv.fit(x_train, y_train)
y_pred = nv.predict(x_test)

# selesai

In [6]:
x_test

,jk,p_mengulang,ipk,p_cuti,sam_bekerja,a_papua,ipk_prodi,ipk1,ipk2,ipk3,ipk4,ipk5,ipk6
7,0,0,0,0,1,0,0,1,1,1,1,1,1
2,0,0,1,1,1,1,1,1,1,1,1,1,1
1,1,1,1,0,1,1,1,0,0,0,0,0,0


In [ ]:
ubah_ke_angka = preprocessing.LabelEncoder()

In [ ]:
le = preprocessing.LabelEncoder()

In [ ]:
encoding = le.fit_transform(y_pred)

In [ ]:
y_pred

In [ ]:
encoding

In [ ]:
le.inverse_transform(encoding)

In [ ]:
y_pred

In [ ]:
encoding

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
x_train.ndim

In [ ]:
x_train.insert(7, 'prediksi', y_train)

In [ ]:
x_train.drop(['prediksi'], axis=1)

In [ ]:
x_train

In [ ]:
x_train

In [ ]:
x_train

In [ ]:
x_train

In [ ]:
dd = x_train.insert(7, 'prediksi', y_train)

In [ ]:
dd

In [ ]:
x_train

In [ ]:
gabung = x_train.copy

In [ ]:
gabung

In [ ]:
x_train

In [ ]:
import copy

In [ ]:
copy

In [ ]:
ba = copy.copy(x_train)

In [ ]:
ba

In [ ]:
ba.insert(7,'prediksi', y_train)

In [ ]:
ba

In [ ]:
x_train

In [ ]:
dg = x_train.copy()

In [ ]:
dg

In [ ]:
dg.insert(7,'prediksi', y_train)

In [ ]:
dg

In [ ]:
x_train

In [ ]:
y_test

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
kenyataan = label_encoder.fit_transform(y_test)		# y_true

In [ ]:
prediksinya = label_encoder.fit_transform(y_pred)	# y_pred

In [ ]:
kenyataan

In [ ]:
prediksinya

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
listnya = ['t', 'tw']

In [ ]:
le.fit(listnya)

In [ ]:
ddf = le.transform(y_pred)

In [ ]:
ddk = le.transform(y_test)

In [ ]:
ddf

In [ ]:
ddk

In [ ]:
a = 3.55354

In [ ]:
round(a, 1)

In [ ]:
a

# evaluasi

In [3]:
from sklearn.metrics import precision_score

In [4]:
data_aktual = [0, 1, 2, 0, 1, 2]
data_prediksi = [0, 2, 1, 0, 0, 1]

In [6]:
precision_score(data_aktual, data_prediksi, average='weighted') 

0.2222222222222222

In [9]:
precision_score(data_aktual, data_prediksi, average=None) 

array([0.66666667, 0.        , 0.        ])

In [ ]:


>>> y_true = [0, 1, 2, 0, 1, 2]
>>> y_pred = [0, 2, 1, 0, 0, 1]
>>> precision_score(y_true, y_pred, average='macro')  
0.22...
>>> precision_score(y_true, y_pred, average='micro')  
0.33...
>>> precision_score(y_true, y_pred, average='weighted')
... 
0.22...
>>> precision_score(y_true, y_pred, average=None)  
array([ 0.66...,  0.        ,  0.        ])

In [14]:
load_data = pd.read_csv('d:/ujii.csv')

In [15]:
from sklearn import preprocessing

In [16]:
le = preprocessing.LabelEncoder()

In [22]:
def ubah(x):
    return le.fit(x)

In [24]:
le.fit(load_data['aktual'])

LabelEncoder()

In [30]:
baru = load_data.copy()

In [28]:
def ubahh(x):
    if x == 'Terlambat':
        return 0
    else:
        return 1

In [39]:
def berhasil(x):
    if x == 'Terlambat':
        return 1
    else:
        return 0

In [69]:
satunol = baru.applymap(berhasil)

In [44]:
y_true = satunol['aktual']
y_pred = satunol['prediksi']

In [49]:
precision_score(y_true, y_pred, average=None)

array([0.95, 0.9 ])

In [53]:
recc = metrics.recall_score(y_true, y_pred, average=None)

In [64]:
def ubah_tipenya(x):
	return x.astype('float')

In [74]:
def uu(x):
    ah = x.astype('float')
    return ah

In [75]:
satunol.applymap(uu)

AttributeError: 'int' object has no attribute 'astype'

In [ ]:
def rru(x)
    return x.astype('float')

In [79]:
satunol.astype('float')

,aktual,prediksi
0,1.0,1.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,1.0,1.0
7,0.0,0.0
8,0.0,0.0
9,0.0,0.0
